Movie Example: https://stackabuse.com/creating-a-simple-recommender-system-in-python-using-pandas

Dataset: https://github.com/zygmuntz/goodbooks-10k
goodreads API: https://www.goodreads.com/api

In [1]:
import pandas as pd

ratings_data = pd.read_csv("https://github.com/zygmuntz/goodbooks-10k/raw/master/ratings.csv")
books_data = pd.read_csv("https://github.com/zygmuntz/goodbooks-10k/raw/master/books.csv")
book_ratings = pd.merge(ratings_data, books_data[['book_id','original_title']], on='book_id')

#books_data.head()
book_ratings.head()
#book_ratings[book_ratings.book_id == 1]
#book_ratings[book_ratings.original_title == 'The Hunger Games']

,user_id,book_id,rating,original_title
0,1,258,5,La sombra del viento
1,11,258,3,La sombra del viento
2,143,258,4,La sombra del viento
3,242,258,5,La sombra del viento
4,325,258,4,La sombra del viento


In [2]:
ratings_mean_count = pd.DataFrame(book_ratings.groupby('original_title')['rating'].mean())
ratings_mean_count['rating_counts'] = pd.DataFrame(book_ratings.groupby('original_title')['rating'].count())
user_book_rating = book_ratings.pivot_table(index='user_id', columns=['original_title'], values='rating')

ratings_mean_count.head()
#user_book_rating.head()

,rating,rating_counts
original_title,,
,3.964730,964
A Monster Calls,4.365180,942
Animal Farm & 1984,4.128205,1404
"Burned (Burned, #1)",4.069697,330
Call the midwife : a true story of the East End in the 1950s,4.134228,447


In [3]:
hunger_games_ratings = user_book_rating['The Hunger Games']
books_like_hunger_games = user_book_rating.corrwith(hunger_games_ratings)

hunger_games_ratings.head()
#books_like_hunger_games.head()

user_id
1   NaN
2   NaN
3   NaN
4   NaN
5   NaN
Name: The Hunger Games, dtype: float64

In [4]:
corr_hunger_games = pd.DataFrame(books_like_hunger_games, columns=['Correlation'])
corr_hunger_games.dropna(inplace=True)
corr_hunger_games.head()

,Correlation
original_title,
,0.197635
A Monster Calls,0.103923
Animal Farm & 1984,0.036472
"Burned (Burned, #1)",0.195531
Call the midwife : a true story of the East End in the 1950s,0.115430


In [5]:
corr_hunger_games = corr_hunger_games.merge(ratings_mean_count['rating_counts'], on='original_title')
corr_hunger_games[corr_hunger_games['rating_counts']>1000].sort_values('Correlation', ascending=False).head(10)

,Correlation,rating_counts
original_title,,
The Hunger Games,1.000000,22806
The Hunger Games Box Set,0.684692,1144
Catching Fire,0.669657,16549
Mockingjay,0.533155,15953
Divergent,0.371507,11780
Moon Called,0.356674,1170
Twilight,0.330199,17657
Song of Susannah,0.318022,1572
The Amityville Horror,0.309538,1066
